# Algoritmo final: Decision Tree

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import datetime

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
inicio = datetime.datetime.now()

In [130]:
propiedades = pd.read_csv('../../set_datos_sup_desc.csv')
set_pruebas = pd.read_csv("../properati_dataset_modificado_descripciones.csv")

In [131]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

propiedades.loc[:, 'Year'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[0]))

propiedades.loc[:, 'Month'] = propiedades.loc[:,'created_on'].apply(lambda x: int(x.split('-')[1]))


propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull())\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min) &\
                              (propiedades.Year >=2017),:]


# Columnas

In [132]:
col = ['superficie','lat', 'lon','property_type_encoded']

comodidades = ['seguridad','gimnasio', 'aire', 'pileta', 'cochera','transporte','cocina','comedor',\
'living', 'banos', 'balcon','ambientes','departamento', 'dormitorios','quincho','porcelanato',\
'completo', 'edificio','lavadero', 'amplio', 'planta', 'cuadras', 'placard', 'terraza', 'acceso',\
'parrilla', 'luminoso','madera', 'credito', 'calidad', 'cucicba', 'amenities','expensas','mesada',\
'responsable','muebles', 'diario', 'independiente', 'cubierta','contrafrente', 'integrada',\
'recepcion','categoria', 'profesional', 'baulera', 'natural','unidades', 'principal', 'espacio', 'estacion', 'vestidor',\
'solarium', 'central','parquet', 'estrenar', 'granito','patio',  'casa', 'suite', 'vista', 'hall', 'agua',  \
'lote', 'fondo', 'acero', 'torre',  'horno',  'sala', 'salon', 'frente','toilette','piso', 'pisos', 'apto',\
'sobre','doble','marca','nuevo', 'bacha', 'ideal','excelente']

len(comodidades)

81

# Parametros del estimador

In [133]:
criterion = 'mse'
max_depth = 20
max_features = 0.8

# Con un set de validacion

In [117]:
columnas = col + comodidades
X_train, X_test, y_train, y_test = train_test_split(propiedades.loc[:,columnas+['description']], \
                                        propiedades.loc[:,'price_aprox_usd'], test_size=0.2)
X_train = pd.DataFrame(X_train).reset_index(drop = True)
X_test = pd.DataFrame(X_test).reset_index(drop = True)

In [118]:
tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
tree.fit(X_train.loc[:,columnas],y_train)
a = tree.predict(X_test.loc[:,columnas])
precision = tree.score(X_test.loc[:,columnas],y_test) * 100
error = mean_squared_error(tree.predict(X_test.loc[:,columnas]), y_test)
print("Precision = {:.2f} %,  Error = {:.0f}".format(precision,error))

Precision = 93.49 %,  Error = 2487568555


# Con el set de pruebas

In [135]:
set_pruebas.loc[:,'price_usd'] = 0.0
cant = 50

n = int(len(comodidades) * 0.8)

for i in range(cant):
    com = list(np.random.choice(comodidades,n,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    tree = DecisionTreeRegressor(criterion = criterion, max_depth = max_depth, max_features = max_features)
    
    tree.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + \
                set_pruebas.loc[:,columnas].apply(lambda x: tree.predict([x])[0],axis = 1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [136]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/AlgoritmoFinal_clase.csv', index = False)

In [11]:
fin = datetime.datetime.now()
tiempo = fin - inicio
print("Tardo {:.0f} minutos.".format(tiempo.seconds / 60))

Tardo 2 minutos.
